In [8]:
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm import tqdm
import pandas as pd
import time
import csv
import os

## Funzione per trasformare i dati in CSV
Lo script prende in input un file .txt con il parametro 'rb' quindi in lettura in formato bytecode che vuol dire che dovrà essere decodificato.
Per prima cosa si scorre fino alla riga che inizia con 'NR' perché i dati iniziano da li, quando la trovo imposto flag_data a 1 che serve a dirmi che da li in poi dovrò salvare le righe.
Cerco la prima riga che inizia con '0' e la salto settando firsr_row a 1.
Quando sia data_flag che first_row sono a 1 allora inizio a salvare le righe.

Per salvare le righe:
* prima le decodifico con .decode()
* poi le divido con .split() che se usato senza parametri splitta la stringa in base agli spazi
* unisco i valori delle ultime due colonne perché fanno entrambi riferimento alla data
* elimino l'ultima riga perché l'ho unita alla penultima
* faccio un append della riga


In [6]:
def txt_to_csv(filename, filedest):
    
    
    if os.path.isfile(filedest + '.csv'):
        os.remove(filedest + '.csv')
        
    file1 = open(filename + '.txt', "rb")
    file_csv = open(filedest + '.csv', 'w+', newline ='')

    count = 0
    flag_data = 0

    with file_csv:
        for line in file1:

            if b'\xb0C' in line:     # se trovo il carattere °C lo ignoro
                pass
            else:
                line = line.decode() # altrimenti decodifico la riga

                # cerco la riga con i nomi della tabella
                if line.startswith('NR'):
                    flag_data = 1                  # quando trovo i nomi delle colonne imposto flag_data = 1
                    table_names = line.split()     # divido i nomi delle colonne      
                    table_names.remove('LastLine') # elimino 'LastLine' che fa riferimento ad un singolo dato duplicato
                    #for item in table_names: print(item)
                    write = csv.writer(file_csv)  
                    write.writerow(table_names)
                    #print(table_names)

                # Se sono arrivato alla tabella ma la riga inizia con 0 la salto
                elif flag_data == 1 and line.startswith('0'):
                    pass

                # Se sono arrivato alla tabella (flag_data == 1) e non sono alla prima riga
                # inizio a salvare i dati
                elif flag_data == 1:
                    row = line.split()

                    if not line.startswith('[END]') and len(row) == len(table_names)+1: # le righe con meno di 25 elementi hanno dati mancanti
                        row[-2] = str(row[-2]) + ' ' + str(row[-1]) # unisco l'ora col giorno
                        del row[-1] # elimino la colonna con i giorni
                        write = csv.writer(file_csv)
                        write.writerow(row)
                        count = count+1      # aggiorno il contatore delle righe

    #print(str(count) + ' righe salvate.')

#txt_to_csv(filename = 'table/DiagnosisTabl',
#           filedest = 'table/csv/DiagnosisTabl')

Per convertire tutti i file da txt a csv mi basta iterare all'interno della cartella e passare ogni file alla funzione txt_to_csv che prende in input il nome del file da convertire e il percorso di destinazione del file.

In [9]:

# controllo se la cartella 'csv' è presente. Se c'è la elimino e la ricreo, se non c'è la creo.
if os.path.isdir('table/csv'):
    shutil.rmtree('table/csv')
    os.mkdir('table/csv')
else:
    os.mkdir('table/csv')

for filename in os.listdir('table/'):
    if filename.endswith(".txt"): 
        print(filename[:-4])
        txt_to_csv(filename = 'table/' + filename[:-4],
                   filedest = 'table/csv/' + filename[:-4])

TabLogPostTHS3
TabLogPostTHS4
DiagnosisTabl2
TabLogFMS2_SX
TabLogFMS3_SX
TabLogAntTHS4
DiagnTableBasaments4
DiagnosisTabl3
DiagnosisTable
TabLogFMS3_DX
DiagnosisTable4
TabLogPostTHS
TabLogFMS2_DX
DiagnosisTabl4
DiagnTableBasaments
TabLogFMS4_SX
TabLogAntTHS
TabLogAntTHS2
DiagnosisTable3
DiagnTableBasaments3
TabLogPostTHS2
TabLogFMS_DX
TabLogFMS4_DX
LONG_TERM_TEMP
TabLogFMS_SX
DiagnTableBasaments2
DiagnosisTable2
TabLogAntTHS3


## Funzione per caricare il CSV in un DataFrame
La funzione prende in input il nome del file .csv da caricare, converte la colonna DataTime in datetime e la imposta come indice del dataframe. Controlla se il dataframe è ordinato e se non lo è lo riordina. Elimina la colonna NR. Restituisce il dataframe.

In [23]:
def load_data(file):
    #column_subset = ['t_Mach',   't_HydrTa', 't_TTlubr', 't_TT_ab1', 
                     #'t_TT_ab2', 't_HdStUp', 't_HdStLw', 't_Sfrnt1', 
                     #'t_Sfrnt2', 't_Srear',  't_Smotor', 'DataTime']

    df = pd.read_csv (file,
                      #usecols=column_subset,
                      index_col=False)

    df['DataTime'] = pd.to_datetime(df['DataTime'], format='%H:%M:%S %d.%m.%Y')
    
    df.set_index('DataTime', drop = True, inplace=True)
    
    if not df.index.is_monotonic:
        df.sort_index(inplace = True)
    
    df.drop('NR', axis=1, inplace=True)
    
    return df

## Processo per caricare tutti i file dello stesso tipo e salvarli come unico csv

In [10]:
def create_single_csv(path_folder, tab_name):
    data_list=[]

    for filename in os.listdir(path_folder):
        if tab_name in filename: 
            data = load_data(path_folder + 'csv/' + filename[:-4] + '.csv')
            data_list.append(data)

    data_all = pd.concat(data_list)
    data_all.sort_index(inplace = True)
    data_all.to_csv(path_folder + '/csv/' + tab_name + '_all.csv')


create_single_csv('table/', 'DX')

 * Faccio una lista con tutti i nomi dei file ma senza il numero progressivo alla fine
 * Per ogni nome in lista cerco le tabelle con quel nome e le riunisco creando un unico file csv

In [14]:
import os
from tqdm import tqdm
import shutil

# controllo se la cartella 'csv' è presente. Se c'è la elimino e la ricreo, se non c'è la creo.
if os.path.isdir('table/csv'):
    shutil.rmtree('table/csv')
    os.mkdir('table/csv')
else:
    os.mkdir('table/csv')

#per ogni file .txt creo il corrispettivo in .csv nella cartella table/csv
table_list = []
pbar = tqdm(os.listdir('table/'), desc='converto .txt in .csv\t')
for filename in pbar:
    if filename.endswith(".txt"):
        #t.set_description("Bar desc (file %i)" % i)
        print(filename[:-4])
        txt_to_csv(filename = 'table/' + filename[:-4],
                   filedest = 'table/csv/' + filename[:-4])
        
        # creo una lista con i nomi delle tabelle da unire
        if not filename[-5:-4].isdigit():
            table_list.append(filename[:-4])

# unisco i csv
pbar = tqdm(table_list, desc='unisco le tabelle\t')
for table in pbar:
    create_single_csv('table/', table)

# elimino i file originali e tengo solo i .csv uniti
pbar = tqdm(os.listdir('table/csv'), desc='elimino tab divise\t')
for filename in pbar:
    if not filename.endswith('_all.csv'):
        os.remove('table/csv/'+filename)
        
print(table_list)

converto .txt in .csv	:   2%|▎         | 1/40 [00:00<00:07,  5.30it/s]

TabLogPostTHS3
TabLogPostTHS4


converto .txt in .csv	:  12%|█▎        | 5/40 [00:00<00:03, 10.41it/s]

DiagnosisTabl2
TabLogFMS2_SX


converto .txt in .csv	:  18%|█▊        | 7/40 [00:00<00:04,  7.67it/s]

...

DiagnTableBasaments2
DiagnosisTable2


converto .txt in .csv	:  90%|█████████ | 36/40 [00:05<00:00,  6.77it/s]

TabLogAntTHS3


elimino tab divise	: 100%|██████████| 41/41 [00:00<00:00, 2947.26it/s]

['TabLogFMS2_SX', 'TabLogFMS3_SX', 'DiagnosisTable', 'TabLogFMS3_DX', 'TabLogPostTHS', 'TabLogFMS2_DX', 'DiagnTableBasaments', 'TabLogFMS4_SX', 'TabLogAntTHS', 'TabLogFMS_DX', 'TabLogFMS4_DX', 'LONG_TERM_TEMP', 'TabLogFMS_SX']


In [74]:
create_single_csv('table/', 'TabLogFMS_DX')
df = pd.read_csv ('table/csv/TabLogFMS_DX_all.csv', index_col='DataTime')
df.index = pd.to_datetime(df.index)
df.plot()

FileNotFoundError: [Errno 2] No such file or directory: 'table/csv/TabLogFMS_DX_all.csv'

## Caricamento dati in modo manuale

In [38]:
def CARICA_TABELLE_BRUTTO():

    #--- carico tutti i file TabLogFMS_DX
    dflist=[]
    for file in os.listdir('table/csv'):
        if 'TabLogFMS' in file and '_DX' in file and not '_all' in file:
            print(file)
            df = load_data('table/csv/' + file)
            dflist.append(df)

    data_all = pd.concat(dflist)
    data_all.sort_index(inplace = True)
    data_all.to_csv('table/csv/TabLogFMS_DX_all.csv')    

    #--- carico tutti i file TabLogFMS_SX
    dflist=[]
    for file in os.listdir('table/csv'):
        if 'TabLogFMS' in file and '_SX' in file and not '_all' in file:
            print(file)
            df = load_data('table/csv/' + file)
            dflist.append(df)

    data_all = pd.concat(dflist)
    data_all.sort_index(inplace = True)
    data_all.to_csv('table/csv/TabLogFMS_SX_all.csv')

    #--- carico tutti i file TabLogPostTHS
    dflist=[]
    filename = 'TabLogPostTHS'
    for file in os.listdir('table/csv'):
        if filename in file and not '_all' in file:
            print(file)
            df = load_data('table/csv/' + file)
            dflist.append(df)

    data_all = pd.concat(dflist)
    data_all.sort_index(inplace = True)
    data_all.to_csv('table/csv/' + filename + '_all.csv')

    #--- carico tutti i file DiagnosisTabl
    dflist=[]
    filename = 'DiagnosisTabl'
    for file in os.listdir('table/csv'):
        if filename in file and not '_all' in file:
            print(file)
            df = load_data('table/csv/' + file)
            dflist.append(df)

    data_all = pd.concat(dflist)
    data_all.sort_index(inplace = True)
    data_all.to_csv('table/csv/' + filename + '_all.csv')

    #--- carico tutti i file TabLogAntTHS
    dflist=[]
    filename = 'TabLogAntTHS'
    for file in os.listdir('table/csv'):
        if filename in file and not '_all' in file:
            print(file)
            df = load_data('table/csv/' + file)
            dflist.append(df)

    data_all = pd.concat(dflist)
    data_all.sort_index(inplace = True)
    data_all.to_csv('table/csv/' + filename + '_all.csv')

    #--- carico tutti i file DiagnTableBasaments
    dflist=[]
    filename = 'DiagnTableBasaments'
    for file in os.listdir('table/csv'):
        if filename in file and not '_all' in file:
            print(file)
            df = load_data('table/csv/' + file)
            dflist.append(df)

    data_all = pd.concat(dflist)
    data_all.sort_index(inplace = True)
    data_all.to_csv('table/csv/' + filename + '_all.csv')

    #--- carico tutti i file LONG_TERM_TEMP
    dflist=[]
    filename = 'LONG_TERM_TEMP'
    for file in os.listdir('table/csv'):
        if filename in file and not '_all' in file:
            print(file)
            df = load_data('table/csv/' + file)
            dflist.append(df)

    data_all = pd.concat(dflist)
    data_all.sort_index(inplace = True)
    data_all.to_csv('table/csv/' + filename + '_all.csv')

# elimino i file originali e tengo solo i .csv uniti che carico in un dizionario di dataframe
dataDict={}
for filename in os.listdir('table/csv'):
    if not filename.endswith('_all.csv'):
        os.remove('table/csv/'+filename)
    elif filename.endswith('_all.csv'):
        print(filename)
        df = pd.read_csv ('table/csv/'+filename, index_col='DataTime')
        df.index = pd.to_datetime(df.index)
        df = df.resample('5T').mean()
        dataDict[filename[:-4]] = df


TabLogFMS3_DX.csv
TabLogFMS_DX.csv
TabLogFMS4_DX.csv
TabLogFMS2_DX.csv
TabLogFMS_SX.csv
TabLogFMS3_SX.csv
TabLogFMS4_SX.csv
TabLogFMS2_SX.csv
TabLogPostTHS.csv
TabLogPostTHS3.csv
TabLogPostTHS2.csv
TabLogPostTHS4.csv
DiagnosisTabl3.csv
DiagnosisTabl2.csv
DiagnosisTable3.csv
DiagnosisTable.csv
DiagnosisTable4.csv
DiagnosisTabl4.csv
DiagnosisTable2.csv
TabLogAntTHS4.csv
TabLogAntTHS3.csv
TabLogAntTHS.csv
TabLogAntTHS2.csv
DiagnTableBasaments2.csv
DiagnTableBasaments3.csv
DiagnTableBasaments4.csv
DiagnTableBasaments.csv
LONG_TERM_TEMP.csv
TabLogAntTHS_all.csv
DiagnTableBasaments_all.csv
TabLogFMS_SX_all.csv
LONG_TERM_TEMP_all.csv
TabLogPostTHS_all.csv
DiagnosisTabl_all.csv
TabLogFMS_DX_all.csv
TabLogTHS_all.csv


In [59]:
dataDict={}
for filename in os.listdir('table/csv'):
    print(filename)
    df = pd.read_csv ('table/csv/'+filename, index_col='DataTime')
    dataDict[filename[:-4]] = df


dataDict['TabLogAntTHS_all'].head()

TabLogAntTHS_all.csv
TabLogFMS2_DX_all.csv
DiagnTableBasaments_all.csv
TabLogFMS_SX_all.csv
DiagnosisTable_all.csv
TabLogFMS2_SX_all.csv
TabLogFMS4_SX_all.csv
LONG_TERM_TEMP_all.csv
TabLogFMS4_DX_all.csv
TabLogFMS3_SX_all.csv
TabLogPostTHS_all.csv
TabLogFMS3_DX_all.csv
TabLogFMS_DX_all.csv


,t_Mach,t_HydrTa,t_TTlubr,t_TT_ab1,t_TT_ab2,t_HdStUp,t_HdStLw,t_Sfrnt1,t_Sfrnt2,t_Srear,t_Smotor,t_ReqDom,t_ExtDom,t_ProDom,t_Envir
DataTime,,,,,,,,,,,,,,,
2021-04-29 08:23:01,19.4,21.6,19.2,19.2,19.3,20.7,20.4,23.0,23.3,23.6,25.6,20,20,20,21.4
2021-04-29 08:24:01,19.4,21.6,19.2,19.2,19.3,20.7,20.4,23.0,23.3,23.6,25.0,20,20,20,21.4
2021-04-29 08:25:01,19.4,21.6,19.2,19.2,19.3,20.7,20.4,22.9,23.2,23.6,25.6,20,20,20,21.4
2021-04-29 08:26:01,19.4,21.6,19.2,19.2,19.3,20.7,20.4,23.3,23.3,23.6,26.0,20,20,20,21.4
2021-04-29 08:27:01,19.4,21.6,19.2,19.2,19.3,20.7,20.4,23.6,23.5,23.6,26.0,20,20,20,21.4
